In [1]:
"""
Link perimeters to FIRED
"""

import os, sys
import geopandas as gpd

import warnings
warnings.filterwarnings("ignore") # suppresses annoying geopandas warning

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

Ready to go !


In [17]:
# Load the SRM fires with greater than 100 AFD observations
fp = os.path.join(projdir,'data/spatial/mod/NIFC/nifc-ics_2018_to_2023-aspen_SRM.gpkg')
nifc = gpd.read_file(fp)
nifc['START_YEAR'] == nifc['START_YEAR'].astype(int)
nifc.head()

,NIFC_ID,NIFC_NAME,NIFC_ACRES,FINAL_ACRES,pct_aspen,INCIDENT_ID,INCIDENT_NAME,START_YEAR,CAUSE,DISCOVERY_DATE,...,WF_CESSATION_DOY,STR_DESTROYED_TOTAL,STR_DAMAGED_TOTAL,STR_THREATENED_MAX,EVACUATION_REPORTED,PEAK_EVACUATIONS,WF_PEAK_AERIAL,WF_PEAK_PERSONNEL,na_l3name,geometry
0,14,416,54130.98,54129,38.614463,2018_9091030_416,416,2018,U,2018-06-01 11:02:00,...,184,0,0,3386,1,2182,15,429.125,Southern Rockies,"MULTIPOLYGON (((-1047907.942 1672642.863, -104..."
1,24,PLATEAU,19677.32,19634,1.305924,2018_9223810_PLATEAU,PLATEAU,2018,L,2018-07-22 16:18:00,...,229,0,0,NaN,NaN,NaN,4,143.6,Southern Rockies,"POLYGON ((-1094448.086 1686999.388, -1094448.7..."
2,45,BEAR CREEK,1102.97,1102,35.344828,2023_16010727_BEAR CREEK,BEAR CREEK,2023,L,2023-07-28 14:44:00,...,253,0,0,3,NaN,NaN,1,85.83333333333333,Southern Rockies,"MULTIPOLYGON (((-988843.179 1669227.920, -9888..."
3,46,QUARTZ RIDGE,2849.89,2850,4.957256,2023_16087351_QUARTZ RIDGE,QUARTZ RIDGE,2023,L,2023-08-05 10:01:00,...,253,0,0,10,NaN,NaN,1,102,Southern Rockies,"MULTIPOLYGON (((-945927.535 1645291.403, -9459..."
4,48,TRAIL SPRINGS,1358.02,1358,32.453957,2023_16071780_TRAIL SPRINGS,TRAIL SPRINGS,2023,L,2023-10-19 15:11:00,...,311,0,0,10,NaN,0,4,83,Southern Rockies,"POLYGON ((-985814.899 1643184.755, -985817.583..."


In [41]:
# Read in FIRED events
fp = os.path.join(projdir,'data/spatial/raw/FIRED/fired-events_west_2012_to_2023.gpkg')
fired = gpd.read_file(fp)
fired['ig_year'] == fired['ig_year'].astype(int)
fired = fired[fired['ig_year'] >= 2018]
fired.head()

,id,ig_date,ig_day,ig_month,ig_year,last_date,event_dur,tot_pix,tot_ar_km2,fsr_px_dy,fsr_km2_dy,mx_grw_px,mx_grw_km2,mx_grw_dte,ig_utm_x,ig_utm_y,na_l3name,geometry
1,10,2018-10-22,295,10,2018,2018-10-22,1,1,0.214659,1.000000,0.214659,1,0.214659,2018-10-22,4.811734e+06,-1.001288e+07,Coast Range,"MULTIPOLYGON (((-2203843.968 2576124.568, -220..."
2,13,2018-10-20,293,10,2018,2018-10-20,1,1,0.214659,1.000000,0.214659,1,0.214659,2018-10-20,4.807101e+06,-1.003281e+07,Coast Range,"MULTIPOLYGON (((-2217666.497 2575421.997, -221..."
4,15,2023-09-16,259,9,2023,2023-09-16,1,1,0.214659,1.000000,0.214659,1,0.214659,2023-09-16,4.803858e+06,-1.003975e+07,Coast Range,"MULTIPOLYGON (((-2220659.090 2572922.473, -222..."
6,18,2020-09-05,249,9,2020,2020-09-11,7,10,2.146587,1.428571,0.306655,7,1.502611,2020-09-07,4.796445e+06,-1.008748e+07,Coast Range,"MULTIPOLYGON (((-2259133.559 2576373.292, -225..."
7,19,2018-10-20,293,10,2018,2018-10-22,3,2,0.429317,0.666667,0.143106,1,0.214659,2018-10-20,4.795055e+06,-1.004022e+07,Coast Range,"MULTIPOLYGON (((-2211747.539 2561494.787, -221..."


In [42]:
nifc['START_YEAR'].unique()

array(['2018', '2023', '2021', '2020', '2019', '2022'], dtype=object)

In [45]:
# Identify spatial joins for each perimeter, extract dates

fired_ = fired.to_crs(proj)
nifc = nifc.to_crs(proj)

# Filter out small fires from FIRED
fired_ = fired[fired['tot_ar_km2'] > 4.04686] # 100 acres

results = []
for idx, row in nifc.iterrows():
    fire_year = int(row['START_YEAR'])
    
    # Filter FIRED data to the year of the current NIFC fire
    fired_year = fired_[fired_['ig_year'] == fire_year]
    
    # Create a GeoDataFrame for the NIFC fire polygon
    row_gdf = gpd.GeoDataFrame([row], geometry=[row.geometry], crs=nifc.crs)
    
    # Perform spatial join to find intersecting FIRED events
    matches = gpd.sjoin(row_gdf, fired_year, how='inner', predicate='intersects')
    
    # If there are matches, collect the earliest and latest dates
    if not matches.empty:
        ig_date = matches['ig_date'].min()    # Earliest ignition date
        last_date = matches['last_date'].max()  # Latest cessation date
        
        # Append to results
        results.append({'Year': fire_year, 'Earliest_Ignition': ig_date, 'Latest_Cessation': last_date})

# Create a summary DataFrame
fired_summary = pd.DataFrame(results)

# Group by year to get the overall range if needed
fired_summary_by_year = fired_summary.groupby('Year').agg(
    Earliest_Ignition=('Earliest_Ignition', 'min'),
    Latest_Cessation=('Latest_Cessation', 'max')
).reset_index()

fired_summary_by_year['Fire_Year_Days'] = (fired_summary_by_year['Latest_Cessation'] - fired_summary_by_year['Earliest_Ignition']).dt.days + 1

fired_summary_by_year.head(7)

,Year,Earliest_Ignition,Latest_Cessation,Fire_Year_Days
0,2018,2018-05-29,2018-10-24,149
1,2019,2019-09-10,2019-10-09,30
2,2020,2020-08-08,2020-11-17,102
3,2021,2021-05-30,2021-08-08,71
4,2022,2022-04-08,2022-06-29,83


In [46]:
# Load the FIRED date ranges
nifc['DISCOVERY_DATE'] = pd.to_datetime(nifc['DISCOVERY_DATE'])
nifc['WF_CESSATION_DATE'] = pd.to_datetime(nifc['WF_CESSATION_DATE'])

nifc['WF_DURATION'] = (nifc['WF_CESSATION_DATE'] - nifc['DISCOVERY_DATE']).dt.days + 1

active_fires_summary = nifc.groupby('START_YEAR').agg(
    Earliest_Ignition=('DISCOVERY_DATE', 'min'),
    Latest_Cessation=('WF_CESSATION_DATE', 'max')
).reset_index()

active_fires_summary['Fire_Year_Days'] = (active_fires_summary['Latest_Cessation'] - active_fires_summary['Earliest_Ignition']).dt.days + 1

active_fires_summary.head(6)

,START_YEAR,Earliest_Ignition,Latest_Cessation,Fire_Year_Days
0,2018,2018-06-01 11:02:00,2018-10-01 18:00:00,123
1,2019,2019-07-28 13:51:00,2019-10-17 17:30:00,82
2,2020,2020-08-10 14:36:00,2020-11-02 18:00:00,85
3,2021,2021-06-09 16:46:00,2021-07-30 18:00:00,52
4,2022,2022-04-06 17:51:00,2022-06-14 18:00:00,70
5,2023,2023-07-28 14:44:00,2023-11-07 17:00:00,103


In [47]:
# Combine the two:
data = {
    'Year': [2018, 2019, 2020, 2021, 2022, 2023],
    'Earliest_Ignition': [pd.Timestamp('2018-05-29'), pd.Timestamp('2019-07-28'), pd.Timestamp('2020-08-08'), pd.Timestamp('2021-05-30'), pd.Timestamp('2022-04-06'), pd.Timestamp('2023-07-28')],
    'Latest_Cessation': [pd.Timestamp('2018-10-24'), pd.Timestamp('2019-10-17'), pd.Timestamp('2020-11-17'), pd.Timestamp('2021-08-08'), pd.Timestamp('2022-06-29'), pd.Timestamp('2023-11-07')]
}
active_fire_windows = pd.DataFrame(data)
active_fire_windows['Fire_Year_Days'] = (active_fire_windows['Latest_Cessation'] - active_fire_windows['Earliest_Ignition']).dt.days + 1
active_fire_windows.head(6)

,Year,Earliest_Ignition,Latest_Cessation,Fire_Year_Days
0,2018,2018-05-29,2018-10-24,149
1,2019,2019-07-28,2019-10-17,82
2,2020,2020-08-08,2020-11-17,102
3,2021,2021-05-30,2021-08-08,71
4,2022,2022-04-06,2022-06-29,85
5,2023,2023-07-28,2023-11-07,103


In [49]:
# Save this out.
fp = os.path.join(projdir, 'data/tabular/mod/srm-active_fire_windows.csv')
active_fire_windows.to_csv(fp)